In [27]:
import os
import _pickle

from collections import defaultdict

print(os.getcwd())

print(os.listdir(os.getcwd()))

from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import shutil
import math
from collections import defaultdict
import wfdb

import pickle


/Users/Arcrank
['nano.save', 'GenerateRecordObject.ipynb', 'anaconda', '.Rhistory', '.eclipse', '.config', 'Music', 'p032447-2170-04-17-16-46n.pkl', '.bash_profile-anaconda.bak', 'woek.ipynb', '.condarc', 'Untitled1.ipynb', '.pia_manager_crash.log', 'Tidebreak', '.DS_Store', 'nltk_data', '.bash_profile.swm', 'Clinical.ipynb', '.bash_profile.swp', '.CFUserTextEncoding', '.netbeans-derby', '.wget-hsts', 'wekafiles', '.bash_profile.save', 'Untitled3.ipynb', '.subversion', 'Untitled.ipynb', 'Pictures', '.atom', '.utop-history', '.rnd', 'Untitled2.ipynb', '.opam', '.p2', '.ipython', 'Desktop', 'Library', 'eclipse-workspace', '.matplotlib', '.nrn_as2sh', '.lesshst', '.globusonline', '.oracle_jre_usage', '.spyder-py3', 'Function Workspace.ipynb', '.bash_profile.pysave', 'README.md', 'example.ipynb', '.cups', '.bash_sessions', 'PycharmProjects', 'Public', 'Datematching.ipynb', '.idlerc', 'Working Artifact Detection.ipynb', '.RData', '.bash_profile.swn', 'work', 'ArtifactDetectors.ipynb', '.ana

In [28]:
class newRecord:
    def __init__(self, record):

        # Immediately available information
        self.record_name = record['record_name']
        self.fs = record['fs']
        self.counter_freq = record['counter_freq']
        self.sig_len = record['sig_len']
        self.base_time = record['base_time']
        self.base_date = record['base_date']
        self.base_datetime = record['base_datetime']

        # Derived Information

        def getLocation(record):
            """
            Pulls location from comments field
            """
            comments = record['comments']
            for c in comments:
                if c.lower().startswith('location:'):
                    _, location = c.split(':')

                    return (location.strip())
            return None

        self.location = getLocation(record)

        def detectHRArtifact(signal):
            """
            Parameters: < 20
                        > 220
            """

            indices = []

            for index, signalValue in enumerate(signal):

                if signalValue < 20 or signalValue > 220:
                    indices.append(index)
            # print(indices)
            return indices

        def detectABPArtifact(sys, dias, mean, fs):
            """

            NEED TO DETERMINE HOW TO DOWNSAMPLE 1HZ SIGNAL
            Using a similar approach then Sun et al.

            Condition 1: SBP >= 300 or SBP <= 20
            Condition 2: SBP - DBP <= 5
                         SBP - DBP > 200

            Condition 3: DBP <=5 or DBP >= 225

            Condition 4: BP beat-to beat > 20mmhg for Ps,Pd, mean

            Condition 5: += 80 mmhg / min change

            Note: not complete for 1Hz condition 5, need to determine how to downsample to 1/60Hz
            """

            sysIndices = []
            diasIndices = []
            meanIndices = []

            for index, sysSignalValue in enumerate(sys):
                # Condition 1
                if sysSignalValue >= 300 or sysSignalValue <= 20:
                    sysIndices.append(index)
                    # print('sys max')
                # Condition 2
                if sysSignalValue - dias[index] <= 5 or sysSignalValue - dias[index] > 200:
                    # print('cond 2',sysSignalValue,dias[index])
                    sysIndices.append(index)
                    diasIndices.append(index)
                    meanIndices.append(index)
            # Condition 3
            for index, diasSignalValue in enumerate(dias):

                if diasSignalValue <= 5 or diasSignalValue >= 225:
                    # print('dias cond',diasSignalValue)
                    diasIndices.append(index)

            def detectDelta(signal, delta):
                flaggedsig = []
                for index, sigVal in enumerate(signal[:-1]):
                    if not np.isnan(sigVal) and not np.isnan(signal[index + 1]):
                        if abs(signal[index + 1] - sigVal) > delta:
                            # flaggedsig.append(index)
                            flaggedsig.append(index + 1)
                return flaggedsig

            if round(fs, 4) == .0167:
                for index, signal in enumerate(sys[:-1]):
                    # Sys


                    if not np.isnan(signal) and not np.isnan(sys[index + 1]):
                        if abs(sys[index + 1] - signal) > 80:
                            sysIndices.append(index + 1)
                    # Dias
                    if not np.isnan(dias[index]) and not np.isnan(dias[index + 1]):
                        if abs(dias[index + 1] - dias[index]) > 80:
                            diasIndices.append(index + 1)
                    # Mean
                    if not np.isnan(mean[index]) and not np.isnan(mean[index + 1]):
                        if abs(mean[index + 1] - mean[index]) > 80:
                            meanIndices.append(index + 1)

            return sysIndices, diasIndices, meanIndices

        def getSignal(signal, record, signalsDict):

            variations = signalsDict[signal]
            for variation in variations:
                if variation in record['sig_name']:
                    ind = record['sig_name'].index(variation)
                    units = record['units'][ind]
                    return record['p_signal'][:, ind], units

            return None, None

        def startOfSignal(signal):
            for index, value in enumerate(signal):
                if not np.isnan(value):
                    return index

            return None

        def endOfSignal(signal):
            for index, sample in enumerate(signal[::-1]):
                if not np.isnan(sample):
                    return len(signal) - index
            return None

        def processABPSignal(signal, indices):
            # noNans = interpolateNans(signal)
            noArtifacts = artifact_to_nan(signal, indices)
            correctedSignal = interpolateNans(noArtifacts)
            return correctedSignal



            # Signal Processing

        def get_time_frames(signal):

            start = startOfSignal(signal)
            end = endOfSignal(signal)

            timeframes = detectGaps(signal, start, end, self.fs)

            return timeframes

        def get_start_end(signal):
            start = startOfSignal(signal)
            end = endOfSignal(signal)
            return (start, end)

        def artifact_to_nan(vector, indices):
            c = vector.copy()
            for value in indices:
                c[value] = np.nan

            return c

        def replaceNansTails(signal, sig_len, start_end):
            start = start_end[0]
            end = start_end[1]
            outputsignal = signal.copy()
            for i in range(sig_len):

                if i < start or i > end:
                    a = outputsignal[i]
                    outputsignal[i] = np.nan

            return outputsignal

        def interpolateNans(signal):
            def nan_helper(y):
                return np.isnan(y), lambda z: z.nonzero()[0]

            nans, x = nan_helper(signal)
            signal[nans] = np.interp(x(nans), x(~nans), signal[~nans])

            return signal

        def processHRSignal(signal):
            # noNans = interpolateNans(signal)
            noArtifacts = artifact_to_nan(signal, detectHRArtifact(signal))

            correctedSignal = interpolateNans(noArtifacts)
            return correctedSignal




            # self.units = record['units']

        def detectGaps(signal, start, end, fs):

            hourGap = 60 if round(fs, 4) == .0167 else 3600 if fs == 1 else None

            if hourGap is None:
                return "Fatal error wrong frequency"

            frames = []
            cache = 0
            placeholderStart = start
            placeholderEnd = start
            # Only look at frame from start to end of signal, excluding trailing and leading NaNs
            for index, value in enumerate(signal[start:end]):

                if np.isnan(value):
                    # means we are currently in a gap
                    cache += 1
                if not np.isnan(value):
                    # value is present, check if the cache is greater than an hour, if so, add the start and end indices
                    if cache > hourGap:
                        # print('found gap at',index,value)
                        frames.append((placeholderStart, placeholderEnd + 1))
                        cache = 0
                        placeholderStart = index

                    else:
                        placeholderEnd = index
                        cache = 0

            frames.append((placeholderStart, placeholderEnd + 1))

            if frames != []:
                return frames
            else:
                return [(start, end)]

        def nan_out_gaps(signal, goodTimes):
            copy = signal.copy()

            l = len(signal)

            goodSample = []
            for s, e in goodTimes:
                goodSample.append([i for i in range(s, e)])

            goodSamples = [item for sublist in goodSample for item in sublist]

            badIndices = []
            for i in range(l):
                if i not in goodSamples:
                    badIndices.append(i)

            for sample in badIndices:
                copy[sample] = np.nan

            return copy

        #         processSignal()signal, start/end/timeframes,arfitactdetector

        #         interpolateNans():
        #         detectABPartifact
        #         interpolateNans():
        #         Nanout gaps
        #         nan out tails
        #         return signal

        #         def processSignal(signal,start_end,timeframes,artifact_alg):
        #             signal = interpolateNans(signal)





        # Get raw signal
        self.rawABPsys, self.ABPsysUnits = getSignal('ABP Sys', record, signalDict)
        self.rawABPdias, self.ABPdiasUnits = getSignal('ABP Dias', record, signalDict)
        self.rawABPmean, self.ABPmeanUnits = getSignal('ABP Mean', record, signalDict)

        self.rawHR, self.HRUnits = getSignal('HR', record, signalDict)

        # get start/end of signal
        self.ABPsys_se = get_start_end(self.rawABPsys)
        self.ABPdias_se = get_start_end(self.rawABPdias)
        self.ABPmean_se = get_start_end(self.rawABPmean)
        self.HR_se = get_start_end(self.rawHR)

        self.ABPsystf = get_time_frames(self.rawABPsys)
        self.ABPdiastf = get_time_frames(self.rawABPdias)
        self.ABPmeantf = get_time_frames(self.rawABPmean)
        self.HRtf = get_time_frames(self.rawHR)

        # interpolate over nans
        #         self.abpsys1 = interpolateNans(self.rawABPsys.copy())
        #         self.abpdias1 = interpolateNans(self.rawABPdias.copy())
        #         self.abpmean1 = interpolateNans(self.rawABPmean.copy())
        self.HR1 = interpolateNans(self.rawHR)

        def processABP(sys, dias, mean, fs):
            abpsys1 = interpolateNans(sys)
            abpdias1 = interpolateNans(dias)
            abpmean1 = interpolateNans(mean)

            # detect artifacts
            sysIndices, diasIndices, meanIndices = detectABPArtifact(abpsys1, abpdias1, abpmean1, fs)

            abpsys2 = processABPSignal(abpsys1, sysIndices)
            abpdias2 = processABPSignal(abpdias1, diasIndices)
            abpmean2 = processABPSignal(abpmean1, meanIndices)

            abpsys2 = nan_out_gaps(abpsys2, self.ABPsystf)
            abpdias2 = nan_out_gaps(abpdias2, self.ABPdiastf)
            abpmean2 = nan_out_gaps(abpmean2, self.ABPmeantf)

            ABPsys = replaceNansTails(abpsys2, self.sig_len, self.ABPsys_se)
            ABPdias = replaceNansTails(abpdias2, self.sig_len, self.ABPdias_se)
            ABPmean = replaceNansTails(abpmean2, self.sig_len, self.ABPmean_se)
            return ABPsys, ABPdias, ABPmean

        self.ABPsys, self.ABPdias, self.ABPmean = processABP(self.rawABPsys.copy(), self.rawABPdias.copy(),
                                                             self.rawABPmean.copy(), self.fs)

        ######### HR #########


        self.HR = processHRSignal(self.rawHR)


        ######################

    #         self.ABP= [(self.ABPsystf,self.ABPsys),(self.ABPdiastf,self.ABPdias),(self.ABPmeantf,self.ABPmean)]

    #     def __setstate__(self, state):
    #         self.__dict__.update(state)

    def printName(self):
        print(self.record_name)
        return None

    def printHR(self):
        for x in self.HRtf:
            s, e = x



In [29]:
proc_file_directory = './Documents/Work/mimic3wdb/processed'
print(os.listdir(proc_file_directory))
recordFiles = os.listdir(proc_file_directory)

['p032805-2182-12-11-09-32n.pkl', 'p032438-2156-03-21-14-53n.pkl', 'p032805-2183-02-28-04-42n.pkl', 'p031648-2107-06-14-06-47n.pkl', 'p032805-2182-12-10-17-19n.pkl', 'p032632-2109-04-03-13-40n.pkl', 'p032805-2182-12-26-14-41n.pkl', 'p031624-2118-08-04-11-11n.pkl', 'p032436-2130-06-20-18-39n.pkl', 'p031683-2112-06-02-18-46n.pkl', 'p032438-2156-03-23-14-13n.pkl', 'p032436-2130-06-20-18-41n.pkl', 'p031285-2184-02-18-14-13n.pkl', 'p032632-2109-04-01-20-06n.pkl', 'p032293-2145-02-11-21-46n.pkl', 'p032436-2130-06-21-17-40n.pkl', 'p032805-2182-12-13-10-11n.pkl', 'p032064-2133-04-28-17-49n.pkl', 'p032805-2182-12-20-13-02n.pkl', 'p030733-2188-08-02-12-39n.pkl', 'p032805-2182-12-13-10-00n.pkl', 'p032805-2182-12-11-10-46n.pkl']


In [30]:
signalNames = ['ABPsys','ABPdias','ABPmean','HR']

signalLengths = defaultdict(list)
def trySignal(sig,rec):
    sigfield = sig+'tf'
    try:
        vals = rec[sigfield]
        print(vals)
        l = []
        for s,e in vals:
            l.append(e-s)
            
        return l
    except:
        return 
    

for r in recordFiles:
    filepath = os.path.join(proc_file_directory,r)
    #print(filepath)
    
    with open(filepath, 'rb') as f:
        record = pickle.load(f)
        
        rd = record.__dict__

        for sig in signalNames:
            a = trySignal(sig,rd)
            if a is not None and trySignal(sig,rd)
            signalLengths[sig].append(trySignal(sig,rd))
            
        
        f.close()
            
    
    
for s in signalLengths:
    print(s,signalLengths[s])

{'record_name': 'p032805-2182-12-11-09-32n', 'fs': 1, 'counter_freq': None, 'sig_len': 57, 'base_time': datetime.time(9, 32), 'base_date': datetime.date(2182, 12, 11), 'base_datetime': datetime.datetime(2182, 12, 11, 9, 32), 'location': 'ccu', 'rawABPsys': None, 'ABPsysUnits': None, 'rawABPdias': None, 'ABPdiasUnits': None, 'rawABPmean': None, 'ABPmeanUnits': None, 'rawHR': array([ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  92.,  92.,  92.,  92.,  92.,  92.,  92.,  92.,  92.,
        92.,  92.,  92.,  92.,  92.,  92.,  92.,  92.,  92.,  92.,  92.,
        92.,  92.,  92.,  92.,  92.,  92.,  92.,  92.,  92.,  92.,  92.,
        92.,  92.,  92.,  92.,  92.,  92.,  92.,  92.,  92.,  92.,  92.,
        92.,  92.]), 'HRUnits': 'bpm'}
p032805-2182-12-11-09-32n
{'record_name': 'p032438-2156-03-21-14-53n', 'fs': 1, 'counter_freq': None, 'sig_len': 166056, 'base_time': datetime.time(14, 53), 'base_date': datetime.date(2156, 3, 21), 'base_datetime': date

{'record_name': 'p032293-2145-02-11-21-46n', 'fs': 1, 'counter_freq': None, 'sig_len': 177379, 'base_time': datetime.time(21, 46, 44), 'base_date': datetime.date(2145, 2, 11), 'base_datetime': datetime.datetime(2145, 2, 11, 21, 46, 44), 'location': 'micu/sicu', 'rawABPsys': None, 'ABPsysUnits': None, 'rawABPdias': None, 'ABPdiasUnits': None, 'rawABPmean': None, 'ABPmeanUnits': None, 'rawHR': array([ 115.,   nan,   nan, ...,   nan,   nan,   nan]), 'HRUnits': 'bpm'}
p032293-2145-02-11-21-46n
{'record_name': 'p032436-2130-06-21-17-40n', 'fs': 1, 'counter_freq': None, 'sig_len': 3001, 'base_time': datetime.time(17, 40, 11), 'base_date': datetime.date(2130, 6, 21), 'base_datetime': datetime.datetime(2130, 6, 21, 17, 40, 11), 'location': 'micu/sicu', 'rawABPsys': None, 'ABPsysUnits': None, 'rawABPdias': None, 'ABPdiasUnits': None, 'rawABPmean': None, 'ABPmeanUnits': None, 'rawHR': array([ nan,  nan,  nan, ...,  98.,  98.,  98.]), 'HRUnits': 'bpm'}
p032436-2130-06-21-17-40n
{'record_name': 'p

In [42]:
for sig in signalLengths:
    for i in signalLengths[sig]:
        if i is not None and int(i) > 0:
            print(i)
        


TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'

In [31]:
f = './Documents/Work/mimic3wdb/matched_numeric/p03/p031683/p031683-2112-06-02-18-46n'
read = wfdb.rdrecord(f)
r = read.__dict__

for k in r:
    print(k,r[k])

record_name p031683-2112-06-02-18-46n
n_sig 16
fs 1
counter_freq None
base_counter None
sig_len 196957
base_time 18:46:24
base_date 2112-06-02
comments ['Location: micu/sicu']
sig_name ['HR', 'PULSE', 'ABP', 'ABP Sys', 'ABP Dias', 'ABP Mean', 'NBP', 'NBP Sys', 'NBP Dias', 'NBP Mean', 'RESP', 'SpO2', 'PVC Rate per Minute', 'Rhythm Status', 'Ectopic Status', 'Ectopic Count']
p_signal [[    75.     75.     nan ...,  63004.     nan     nan]
 [    75.     75.     nan ...,  63004.     nan     nan]
 [    75.     75.     nan ...,  63004.     nan     nan]
 ..., 
 [    nan     nan     nan ...,     nan     nan     nan]
 [    nan     nan     nan ...,     nan     nan     nan]
 [    nan     nan     nan ...,     nan     nan     nan]]
d_signal None
e_p_signal None
e_d_signal None
file_name ['3140484n.dat', '3140484n.dat', '3140484n.dat', '3140484n.dat', '3140484n.dat', '3140484n.dat', '3140484n.dat', '3140484n.dat', '3140484n.dat', '3140484n.dat', '3140484n.dat', '3140484n.dat', '3140484n.dat', '31404

In [32]:
signalDict = {
'ABP Dias':['ABPDias','ABP DIAS','ART DIAS','ABP Dias','ART Dias','AOBP Dias','Ao DIAS'],
'ABP Mean':['ABPMean','ABP MEAN','ART MEAN','ABP Mean','ART Mean','AOBP Mean','Ao MEAN'],
'ABP Sys':['ABPSys','ABP SYS','ART SYS','ABP Sys','ART Sys','AOBP Sys','Ao SYS'],
'HR':['HR']
}

def getSignal(signal, record, signalsDict):

    variations = signalsDict[signal]
    for variation in variations:
        if variation in record['sig_name']:
            ind = record['sig_name'].index(variation)
            units = record['units'][ind]
            return record['p_signal'][:, ind], units

    return None, None

abpsys = r['p_signal'][:,3]
for i in abpsys:
    if not np.isnan(i):
        print(i)
#hr = getSignal('HR',r,signalDict)


40.0
40.0
40.0


In [33]:
def startOfSignal(signal):
    for index, value in enumerate(signal):
        if not np.isnan(value):
            return index

    return None

def endOfSignal(signal):
    for index, sample in enumerate(signal[::-1]):
        if not np.isnan(sample):
            return len(signal) - index
    return None

print(startOfSignal(abpsys))

print(endOfSignal(abpsys))

print(abpsys[57330:57336])

57331
57334
[ nan  40.  40.  40.  nan  nan]
